In [1]:
import pandas as pd
import numpy as np
import re
from keras.layers import Dense, Dropout, GRU, Embedding 
from keras.layers import Input, Activation, concatenate, GlobalAveragePooling1D
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.utils import np_utils, get_custom_objects
from keras.preprocessing import text, sequence
from string import ascii_letters, punctuation, digits

def swish(x):
    return (K.sigmoid(x) * x)

get_custom_objects().update({'swish': Activation(swish)})

SEQ_LENGTH = 100
EMBED_SIZE = 300
VOCAB = 238590
np.random.seed(2017)

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### model

In [2]:
def define_model(matrix):
    rnn = {}
    rnn['units'] = 75
    rnn['return_sequences'] = True
    rnn['recurrent_dropout'] = 0.2
    rnn['dropout'] = 0.1
    rnn['activation'] = 'tanh'
    inputs = Input(shape=(SEQ_LENGTH,), name='sequence')
    embed = Embedding(VOCAB,EMBED_SIZE, weights=[matrix], trainable=False)(inputs)
    lstm = Bidirectional(GRU(**rnn))(embed)
    lstm = BatchNormalization()(lstm)
    max_pool = GlobalMaxPooling1D()(lstm)
    avg_pool = GlobalAveragePooling1D()(lstm)
    pool = concatenate([max_pool, avg_pool])
    pool = BatchNormalization()(pool)
    lstm = Dropout(0.2)(pool)
    dense = Dense(200, activation='swish')(lstm)
    dense = Dropout(0.2)(dense)
    predict = Dense(6, activation='sigmoid')(dense)
    model = Model(inputs=[inputs], output=predict)
    optimizer = Adam(lr=1e-3)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

### generator

In [3]:
embeddings_index = {}

f = open('../data/download/glove.6B.300d.txt')

for line in f:
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        print(values[0])
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [4]:
def dataflow(train_text, valid_text, score_text):
    train_text['comment_text'] = train_text['comment_text'].fillna('nan')
    valid_text['comment_text'] = valid_text['comment_text'].fillna('nan')
    score_text['comment_text'] = score_text['comment_text'].fillna('nan')
    train_text = list(train_text['comment_text'].values)
    valid_text = list(valid_text['comment_text'].values)
    score_text = list(score_text['comment_text'].values)
    tokenizer = text.Tokenizer(lower=True, char_level=False, num_words=20000)
    tokenizer.fit_on_texts(train_text + valid_text)
    word_index = tokenizer.word_index
    print('total words:', len(word_index))
    intersect = 0
    embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            intersect += 1
    print('common words:', intersect)
    score_token = tokenizer.texts_to_sequences(score_text)
    score_seq = sequence.pad_sequences(score_token, maxlen=SEQ_LENGTH)
    return score_seq, embedding_matrix

### execute

In [5]:
def score_model(mode):
    train_text = pd.read_csv('../data/data/source_1/train/train_data_{}.csv'.format(mode))
    valid_text = pd.read_csv('../data/data/source_1/train/test_data_{}.csv'.format(mode))
    score_text = pd.read_csv('../data/data/source_1/score/score_text.csv')
    score_data = score_text[['id']]
    score_text, embedding_matrix = dataflow(train_text, valid_text, score_text)
    model = define_model(embedding_matrix)
    path = '../data/data/source_1/model_2/model_{}.hdf5'.format(mode)
    model.load_weights(path)
    scores = model.predict(score_text, batch_size=512)
    scores = pd.DataFrame(scores)
    scores.columns = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
    scores = score_data.join(scores)
    return scores

In [6]:
score_1 = score_model(1)
score_2 = score_model(2)
score_3 = score_model(3)
score_4 = score_model(4)
score_5 = score_model(5)

total words: 238589
common words: 94353


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


total words: 238589
common words: 94353
total words: 238589
common words: 94353
total words: 238589
common words: 94353
total words: 238589
common words: 94353


In [7]:
submit = score_1.append(score_2)
submit = submit.append(score_3)
submit = submit.append(score_4)
submit = submit.append(score_5)
submit = submit.groupby('id').mean().reset_index()
submit.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.985434,0.443933,0.935825,1.125207e-01,0.888198,0.538948
1,0000247867823ef7,0.000093,0.000001,0.000042,2.193530e-08,0.000026,0.000001
2,00013b17ad220c46,0.001082,0.000077,0.001127,9.460520e-07,0.000357,0.000099
3,00017563c3f7919a,0.000262,0.000004,0.000129,1.221449e-05,0.000150,0.000018
4,00017695ad8997eb,0.003484,0.000131,0.001249,4.463878e-05,0.000377,0.000049


In [8]:
submit.to_csv('../data/submit/baseline_2.csv', index=False)

In [ ]:
# LB - 0.9805